# Library

In [ ]:
pip install vncorenlp

     |████████████████████████████████| 2.6 MB 8.3 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=5bde2c558e83fad5a2547fa1c5dfce8989fa20ef88c38ee3aed2e71e02389e06
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp


In [ ]:
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

--2021-11-15 03:09:09--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M   170MB/s    in 0.2s    

2021-11-15 03:09:10 (170 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [27412575/27412575]

--2021-11-15 03:09:10--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting respon

In [ ]:
pip install transformers==4.3.0

     |████████████████████████████████| 1.8 MB 7.4 MB/s 
     |████████████████████████████████| 895 kB 55.6 MB/s 
     |████████████████████████████████| 3.3 MB 34.1 MB/s 


In [ ]:
pip install sentencepiece

     |████████████████████████████████| 1.2 MB 9.8 MB/s 


# Hyper parameters

In [ ]:
MODEL = 'distilbert-base-cased'     # Danh sách pre-trained model ở đây: https://huggingface.co/models
MAX_LEN = 200
BATCH_SIZE = 32
EPOCH = 30
TOKENIZER = 'none'   # Để none nếu không dùng tách từ 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load data

In [ ]:
TRAIN = 'drive/MyDrive/CODE/JobPrediction/dataset/raw_data_new/train.csv'
DEV = 'drive/MyDrive/CODE/JobPrediction/dataset/raw_data_new/dev.csv'
TEST = 'drive/MyDrive/CODE/JobPrediction/dataset/raw_data_new/test.csv'

LABEL = 'drive/MyDrive/CODE/JobPrediction/dataset/raw_data_new/labels.csv'

import pandas as pd

train = pd.read_csv(TRAIN)
dev = pd.read_csv(DEV)
test = pd.read_csv(TEST)

label = pd.read_csv(LABEL)
job_types = label['job_type'].values

In [ ]:
from vncorenlp import VnCoreNLP
import numpy as np
import re

vncorenlp = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

def make_label(data):
    lbl_job = []
    for td in data['job'].values:
        l_job_onehot = np.zeros(len(job_types))
        
        for i in range(0, len(job_types)):
            if job_types[i] in td:
                l_job_onehot[i] = 1
        lbl_job.append(l_job_onehot)

    return lbl_job


def return_label(y):
    lbl_job = []
    for i in range(0, len(y)):
        if y[i] == 1:
            lbl_job.append(job_types[i])

    return lbl_job

def custom_tokenizer(text_data, tokenizer='vncorenlp'):
    # text_data = text_data.lower()
    if tokenizer == 'vncorenlp':
        # return " ".join(vncorenlp.tokenize(str(text_data))[0])
        text = ""
        lst = vncorenlp.tokenize(str(text_data))
        for t in lst:
            text += " ".join(t)
        return text
    if tokenizer == 'none':
        return text_data
    return ViTokenizer.tokenize(str(text_data))

def pre_process(X, y):
    sentences = []
    
    for t in X:
        t = re.sub(r"[-()\"#/@;:<>{}`+=~|!?,]", "", t)
        sentences.append(custom_tokenizer(t, tokenizer=TOKENIZER))

    return (sentences, y)

In [ ]:
y_train = make_label(train)
y_dev = make_label(dev)
y_test = make_label(test)

X_train = train['job_description']
X_dev = dev['job_description']
X_test = test['job_description']

In [ ]:
X1, y1 = pre_process(X_train, y_train)
X2, y2 = pre_process(X_dev, y_dev)
X3, y3 = pre_process(X_test, y_test)

# Model

In [ ]:
from transformers import AutoTokenizer 

tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=False)

train_encodings = tokenizer(X1, truncation=True, padding=True, max_length=MAX_LEN)
dev_encodings = tokenizer(X2, truncation=True, padding=True, max_length=MAX_LEN)
test_encodings = tokenizer(X3, truncation=True, padding=True, max_length=MAX_LEN)

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [ ]:
import tensorflow as tf

train_features = {x: tf.convert_to_tensor(train_encodings[x], dtype=tf.float32) for x in tokenizer.model_input_names}
dev_features = {x: tf.convert_to_tensor(dev_encodings[x], dtype=tf.float32) for x in tokenizer.model_input_names}
test_features = {x: tf.convert_to_tensor(test_encodings[x], dtype=tf.float32) for x in tokenizer.model_input_names}

In [ ]:
from keras.layers import LSTM, Dense, Concatenate, Embedding, Bidirectional, GlobalMaxPooling1D, Dropout, Reshape, GRU, SpatialDropout1D, Conv1D, GlobalAveragePooling1D
from keras.models import Model, Input
import numpy as np

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import Constant
from transformers import TFAutoModelForSequenceClassification, TFBertModel

from tensorflow.keras.losses import BinaryCrossentropy

import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")

units = 100

encoder = TFBertModel.from_pretrained(MODEL)

input_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_ids')
token_type_ids = Input(shape=(MAX_LEN,), dtype=tf.int32, name='token_type_ids')
attention_mask = Input(shape=(MAX_LEN,), dtype=tf.int32, name='attention_mask')
embedding = encoder(
    {
        'input_ids': input_ids, 
        # 'token_type_ids': token_type_ids,
        'attention_mask': attention_mask
        
    }
)[0]

x1 = SpatialDropout1D(0.2)(embedding)

x = Bidirectional(GRU(units, return_sequences = True))(x1)
x = Conv1D(int(units/2), kernel_size = 2, padding = "same", kernel_initializer = "he_uniform")(x)
    
y = Bidirectional(LSTM(units, return_sequences = True))(x1)
y = Conv1D(int(units/2), kernel_size = 2, padding = "same", kernel_initializer = "he_uniform")(y)
    
avg_pool1 = GlobalAveragePooling1D()(x)
max_pool1 = GlobalMaxPooling1D()(x)
    
avg_pool2 = GlobalAveragePooling1D()(y)
max_pool2 = GlobalMaxPooling1D()(y)
    
    
x = Concatenate(axis=-1)([avg_pool1, max_pool1, avg_pool2, max_pool2])
x = Dropout(0.5)(x)
out = Dense(68, activation = "sigmoid")(x)

# model = Model(
#     inputs=[input_ids, token_type_ids, attention_mask],
#     outputs=out,
# )

model = Model(
    inputs=[input_ids, attention_mask],
    outputs=out,
)

loss = BinaryCrossentropy(from_logits=False)
optimizer = Adam(lr=1e-5)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

model.summary()

Downloading:   0%|          | 0.00/354M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-cased were not used when initializing TFBertModel: ['activation_13', 'vocab_transform', 'vocab_projector', 'vocab_layer_norm', 'distilbert']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['bert']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
import warnings
warnings.filterwarnings("ignore")

callback = EarlyStopping(monitor='val_loss', patience=2)

model.fit(train_features, np.array(y1), 
          validation_data=(dev_features, np.array(y2)), 
          batch_size=BATCH_SIZE, epochs=EPOCH)

Epoch 1/30
633/633 [==============================] - 538s 802ms/step - loss: 0.2793 - accuracy: 0.0205 - val_loss: 0.1267 - val_accuracy: 0.0500
Epoch 2/30
633/633 [==============================] - 505s 797ms/step - loss: 0.1779 - accuracy: 0.0313 - val_loss: 0.1226 - val_accuracy: 0.1528
Epoch 3/30
633/633 [==============================] - 511s 808ms/step - loss: 0.1476 - accuracy: 0.0608 - val_loss: 0.1121 - val_accuracy: 0.1949
Epoch 4/30
633/633 [==============================] - 511s 808ms/step - loss: 0.1225 - accuracy: 0.1317 - val_loss: 0.0996 - val_accuracy: 0.2318
Epoch 5/30
633/633 [==============================] - 511s 807ms/step - loss: 0.0995 - accuracy: 0.2422 - val_loss: 0.0925 - val_accuracy: 0.2722
Epoch 6/30
633/633 [==============================] - 511s 807ms/step - loss: 0.0828 - accuracy: 0.3297 - val_loss: 0.0841 - val_accuracy: 0.2886
Epoch 7/30
633/633 [==============================] - 505s 798ms/step - loss: 0.0720 - accuracy: 0.3739 - val_loss: 0.0822 -

# Evaluate

In [ ]:
# Evaluation metric
import sys
import os
import os.path
from scipy.stats import sem
import numpy as np
from ast import literal_eval
import tensorflow as tf
    
def f1(predictions, gold):
    if len(gold) == 0:
        return 1. if len(predictions) == 0 else 0.
    if len(predictions) == 0:
        return 0.
    predictions_set = set(predictions)
    gold_set = set(gold)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)


def evaluate(pred, gold):
    pred_lines = pred
    gold_lines = gold

    # only when the same number of lines exists
    if (len(pred_lines) == len(gold_lines)):
        data_dic = {}
        for n, line in enumerate(gold_lines):
            parts = line.split('\t')
            if len(parts) == 2:
                data_dic[int(parts[0])] = [literal_eval(parts[1])]
            else:
                raise ValueError('Format problem for gold line %d.', n)

        for n, line in enumerate(pred_lines):
            parts = line.split('\t')
            if len(parts) == 2:
                if int(parts[0]) in data_dic:
                    try:
                        data_dic[int(parts[0])].append(literal_eval(parts[1]))
                    except ValueError:
                        # Invalid predictions are replaced by a default value
                        data_dic[int(parts[0])].append([])
                else:
                    raise ValueError('Invalid text id for pred line %d.', n)
            else:
                raise ValueError('Format problem for pred line %d.', n)

        # lists storing gold and prediction scores
        scores = []
        for id in data_dic:
            if len(data_dic[id]) == 2:
                gold_spans = data_dic[id][0]
                pred_spans = data_dic[id][1]
                scores.append(f1(pred_spans, gold_spans))
            else:
                sys.exit('Repeated id in test data.')

        return (np.mean(scores), sem(scores))

def accuracy_score(y_true, y_pred):
    temp = 0
    for i in range(0, len(y_true)):
        temp += sum(np.logical_and(y_true[i], y_pred[i])) / sum(np.logical_or(y_true[i], y_pred[i]))
    return temp / len(y_true)

 
def f1_score(y_true, y_pred):
    temp = 0
    for i in range(len(y_true)):
        if (sum(y_true[i]) == 0) and (sum(y_pred[i]) == 0):
            continue
        temp+= (2*sum(np.logical_and(y_true[i], y_pred[i]))) / (sum(y_true[i])+sum(y_pred[i]))
    return temp/ len(y_true)

def em_score(y_true, y_pred):
    MR = np.all(y_pred == y_true, axis=1).mean()
    return MR

In [ ]:
y_test_pred = model.predict(test_features)
y_dev_pred = model.predict(dev_features)

y_dev_pred_new = []

for y in y_dev_pred:
    lb = []
    for i in range(0, len(y)):
        if y[i] >= 0.5:
            lb.append(1)
        else:
            lb.append(0)
    y_dev_pred_new.append(lb)

y_test_pred_new = []

for y in y_test_pred:
    lb = []
    for i in range(0, len(y)):
        if y[i] >= 0.5:
            lb.append(1)
        else:
            lb.append(0)
    y_test_pred_new.append(lb)

In [ ]:
f1_score(y_dev, y_dev_pred_new)*100, accuracy_score(y_dev, y_dev_pred_new)*100

(57.98579545454518, 54.63839285714302)

In [ ]:
f1_score(y_test, y_test_pred_new)*100, accuracy_score(y_test, y_test_pred_new)*100

(42.17976503254942, 36.46108023682387)

# Error Analysis 

In [ ]:
label_dev = []
label_dev_pred = []
for i in range(len(y_dev)):
    label_dev.append(return_label(y_dev[i]))
    label_dev_pred.append(return_label(y_dev_pred_new[i]))

label_test = []
label_test_pred = []
for i in range(len(y_test)):
    label_test.append(return_label(y_test[i]))
    label_test_pred.append(return_label(y_test_pred_new[i]))

In [ ]:
dev_result = pd.DataFrame([list(X_dev), label_dev, label_dev_pred]).T
test_result = pd.DataFrame([list(X_test), label_test, label_test_pred]).T

In [ ]:
header = ['description', 'true_label', 'predicted_label']
dev_result.columns = header
test_result.columns = header

dev_result.head()